# 泰坦尼克生存率预测

### 项目来自kaggle,代码来自知乎猴子

作者：[杨畅](https://www.kaggle.com/invisibledesigner)

## 预测大纲
1. 采集数据（kaggle已给出）
2. 理解数据
3. 数据清洗
4. 构建模型
5. 模型评估
6. 方案实施
7. 数据可视化
8. 装逼，提高成就感

In [1]:
#忽略警告提醒
#import warnings
#warnings.filterwarnings('ignore')
#数据导入
import numpy as np
import pandas as pd

### 2.1 导入数据

In [2]:
#导入数据
#训练数据
train = pd.read_csv('./train.csv')
#测试数据
test = pd.read_csv('./test.csv')
#观察数据容量
print('训练数据集：', train.shape, '测试数据集', test.shape)

训练数据集： (891, 12) 测试数据集 (418, 11)


In [3]:
rowNum_train = train.shape[0]
rowNum_test = test.shape[0]
print('kaggle训练集有多少行数据：', rowNum_train,
      'kaggle测试集有多少行数据', rowNum_test)

kaggle训练集有多少行数据： 891 kaggle测试集有多少行数据 418


In [4]:
#合并数据集，方便对两个数据集进行清洗
#ignore_index = True 合并时，忽略原有的索引，建立新的索引
full = train.append(test, ignore_index = True)
print("合并后的数据集：", full.shape)

合并后的数据集： (1309, 12)


### 2.2理解数据

In [5]:
#查看数据
full.head()

Age Cabin Embarked     Fare  \
0  22.0   NaN        S   7.2500   
1  38.0   C85        C  71.2833   
2  26.0   NaN        S   7.9250   
3  35.0  C123        S  53.1000   
4  35.0   NaN        S   8.0500   

                                                Name  Parch  PassengerId  \
0                            Braund, Mr. Owen Harris      0            1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...      0            2   
2                             Heikkinen, Miss. Laina      0            3   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)      0            4   
4                           Allen, Mr. William Henry      0            5   

   Pclass     Sex  SibSp  Survived            Ticket  
0       3    male      1       0.0         A/5 21171  
1       1  female      1       1.0          PC 17599  
2       3  female      0       1.0  STON/O2. 3101282  
3       1  female      1       1.0            113803  
4       3    male      0       0.0            373450

In [6]:
full.describe()

Age         Fare        Parch  PassengerId       Pclass  \
count  1046.000000  1308.000000  1309.000000  1309.000000  1309.000000   
mean     29.881138    33.295479     0.385027   655.000000     2.294882   
std      14.413493    51.758668     0.865560   378.020061     0.837836   
min       0.170000     0.000000     0.000000     1.000000     1.000000   
25%      21.000000     7.895800     0.000000   328.000000     2.000000   
50%      28.000000    14.454200     0.000000   655.000000     3.000000   
75%      39.000000    31.275000     0.000000   982.000000     3.000000   
max      80.000000   512.329200     9.000000  1309.000000     3.000000   

             SibSp    Survived  
count  1309.000000  891.000000  
mean      0.498854    0.383838  
std       1.041658    0.486592  
min       0.000000    0.000000  
25%       0.000000    0.000000  
50%       0.000000    0.000000  
75%       1.000000    1.000000  
max       8.000000    1.000000

In [7]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


# 3.数据清洗

## 3.1数据预处理

In [8]:
#年龄和船票价格都填充平均值
full['Age'] = full['Age'].fillna(full['Age'].mean())
full['Fare'] = full['Fare'].fillna(full['Fare'].mean())
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1309 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [9]:
#检查处理数据是否正常
full.head()

Age Cabin Embarked     Fare  \
0  22.0   NaN        S   7.2500   
1  38.0   C85        C  71.2833   
2  26.0   NaN        S   7.9250   
3  35.0  C123        S  53.1000   
4  35.0   NaN        S   8.0500   

                                                Name  Parch  PassengerId  \
0                            Braund, Mr. Owen Harris      0            1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...      0            2   
2                             Heikkinen, Miss. Laina      0            3   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)      0            4   
4                           Allen, Mr. William Henry      0            5   

   Pclass     Sex  SibSp  Survived            Ticket  
0       3    male      1       0.0         A/5 21171  
1       1  female      1       1.0          PC 17599  
2       3  female      0       1.0  STON/O2. 3101282  
3       1  female      1       1.0            113803  
4       3    male      0       0.0            373450

In [10]:
full['Embarked'].head()

0    S
1    C
2    S
3    S
4    S
Name: Embarked, dtype: object

In [11]:
full['Embarked'] = full['Embarked'].fillna('S')

In [12]:
full['Cabin'].head()

0     NaN
1     C85
2     NaN
3    C123
4     NaN
Name: Cabin, dtype: object

In [13]:
#由于缺失值比较多，填充U，表示未知
full['Cabin'] = full['Cabin'].fillna('U')

In [14]:
full.head()

Age Cabin Embarked     Fare  \
0  22.0     U        S   7.2500   
1  38.0   C85        C  71.2833   
2  26.0     U        S   7.9250   
3  35.0  C123        S  53.1000   
4  35.0     U        S   8.0500   

                                                Name  Parch  PassengerId  \
0                            Braund, Mr. Owen Harris      0            1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...      0            2   
2                             Heikkinen, Miss. Laina      0            3   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)      0            4   
4                           Allen, Mr. William Henry      0            5   

   Pclass     Sex  SibSp  Survived            Ticket  
0       3    male      1       0.0         A/5 21171  
1       1  female      1       1.0          PC 17599  
2       3  female      0       1.0  STON/O2. 3101282  
3       1  female      1       1.0            113803  
4       3    male      0       0.0            373450

In [15]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1309 non-null float64
Cabin          1309 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


# 3.2 特征提取

## 性别

In [16]:
full['Sex'].head()

0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

In [17]:
#有直接类别的
sex_mapDict = {'male':1,
               'female':0}
full['Sex'] = full['Sex'].map(sex_mapDict)
full.head()

Age Cabin Embarked     Fare  \
0  22.0     U        S   7.2500   
1  38.0   C85        C  71.2833   
2  26.0     U        S   7.9250   
3  35.0  C123        S  53.1000   
4  35.0     U        S   8.0500   

                                                Name  Parch  PassengerId  \
0                            Braund, Mr. Owen Harris      0            1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...      0            2   
2                             Heikkinen, Miss. Laina      0            3   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)      0            4   
4                           Allen, Mr. William Henry      0            5   

   Pclass  Sex  SibSp  Survived            Ticket  
0       3    1      1       0.0         A/5 21171  
1       1    0      1       1.0          PC 17599  
2       3    0      0       1.0  STON/O2. 3101282  
3       1    0      1       1.0            113803  
4       3    1      0       0.0            373450

## 登船港口

In [18]:
embarkedDf = pd.DataFrame()
embarkedDf = pd.get_dummies(full['Embarked'], prefix = 'Embarked')
embarkedDf.head()

Embarked_C  Embarked_Q  Embarked_S
0           0           0           1
1           1           0           0
2           0           0           1
3           0           0           1
4           0           0           1

In [19]:
#axis = 1即为连接时，横着连接，反之，为0时，则竖着连接
#inpalce 表示当地删除，也就是没有备份
full = pd.concat([full, embarkedDf] , axis = 1)
full.drop('Embarked', axis = 1, inplace = True)
full.head()

Age Cabin     Fare                                               Name  \
0  22.0     U   7.2500                            Braund, Mr. Owen Harris   
1  38.0   C85  71.2833  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2  26.0     U   7.9250                             Heikkinen, Miss. Laina   
3  35.0  C123  53.1000       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4  35.0     U   8.0500                           Allen, Mr. William Henry   

   Parch  PassengerId  Pclass  Sex  SibSp  Survived            Ticket  \
0      0            1       3    1      1       0.0         A/5 21171   
1      0            2       1    0      1       1.0          PC 17599   
2      0            3       3    0      0       1.0  STON/O2. 3101282   
3      0            4       1    0      1       1.0            113803   
4      0            5       3    1      0       0.0            373450   

   Embarked_C  Embarked_Q  Embarked_S  
0           0           0           1  
1           1           0           0  
2           0           0           1  
3           0           0           1  
4           0           0           1

## 客舱等级

In [20]:
pclassDf = pd.DataFrame()
pclassDf = pd.get_dummies(full['Pclass'], prefix = 'Pclass')
pclassDf.head()

Pclass_1  Pclass_2  Pclass_3
0         0         0         1
1         1         0         0
2         0         0         1
3         1         0         0
4         0         0         1

In [21]:
full = pd.concat([full, pclassDf], axis = 1)
full.drop('Pclass', axis = 1, inplace = True)
full.head()

Age Cabin     Fare                                               Name  \
0  22.0     U   7.2500                            Braund, Mr. Owen Harris   
1  38.0   C85  71.2833  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2  26.0     U   7.9250                             Heikkinen, Miss. Laina   
3  35.0  C123  53.1000       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4  35.0     U   8.0500                           Allen, Mr. William Henry   

   Parch  PassengerId  Sex  SibSp  Survived            Ticket  Embarked_C  \
0      0            1    1      1       0.0         A/5 21171           0   
1      0            2    0      1       1.0          PC 17599           1   
2      0            3    0      0       1.0  STON/O2. 3101282           0   
3      0            4    0      1       1.0            113803           0   
4      0            5    1      0       0.0            373450           0   

   Embarked_Q  Embarked_S  Pclass_1  Pclass_2  Pclass_3  
0           0           1         0         0         1  
1           0           0         1         0         0  
2           0           1         0         0         1  
3           0           1         1         0         0  
4           0           1         0         0         1

## 分类数据：字符串类型

In [22]:
#姓名
#根据外国人的姓名特点，提取头衔
def getTitle(name):
    str1 = name.split(',')[1]
    str2 = str1.split('.')[0]
    str3 = str2.strip()
    return str3

In [23]:
#map函数：对Series每个数据应用自定义的函数计算
titleDf = pd.DataFrame()
titleDf['title'] = full['Name'].map(getTitle)
titleDf.head()

title
0    Mr
1   Mrs
2  Miss
3   Mrs
4    Mr

In [24]:
'''
定义以下几种头衔类别：
Officer政府官员
Royalty王室（皇室）
Mr已婚男士
Mrs已婚妇女
Miss年轻未婚女子
Master有技能的人/教师
'''
title_mapDict = {
                    "Capt":       "Officer",
                    "Col":        "Officer",
                    "Major":      "Officer",
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Dr":         "Officer",
                    "Rev":        "Officer",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Mme":        "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Mrs",
                    "Mr" :        "Mr",
                    "Mrs" :       "Mrs",
                    "Miss" :      "Miss",
                    "Master" :    "Master",
                    "Lady" :      "Royalty"
                    }
titleDf['title'] = titleDf['title'].map(title_mapDict)
titleDf = pd.get_dummies(titleDf['title'])
titleDf.head()

Master  Miss  Mr  Mrs  Officer  Royalty
0       0     0   1    0        0        0
1       0     0   0    1        0        0
2       0     1   0    0        0        0
3       0     0   0    1        0        0
4       0     0   1    0        0        0

In [25]:
full = pd.concat([full, titleDf], axis  = 1)
full.drop('Name', axis = 1, inplace = True)
full.head()

Age Cabin     Fare  Parch  PassengerId  Sex  SibSp  Survived  \
0  22.0     U   7.2500      0            1    1      1       0.0   
1  38.0   C85  71.2833      0            2    0      1       1.0   
2  26.0     U   7.9250      0            3    0      0       1.0   
3  35.0  C123  53.1000      0            4    0      1       1.0   
4  35.0     U   8.0500      0            5    1      0       0.0   

             Ticket  Embarked_C   ...     Embarked_S  Pclass_1  Pclass_2  \
0         A/5 21171           0   ...              1         0         0   
1          PC 17599           1   ...              0         1         0   
2  STON/O2. 3101282           0   ...              1         0         0   
3            113803           0   ...              1         1         0   
4            373450           0   ...              1         0         0   

   Pclass_3  Master  Miss  Mr  Mrs  Officer  Royalty  
0         1       0     0   1    0        0        0  
1         0       0     0   0    1        0        0  
2         1       0     1   0    0        0        0  
3         0       0     0   0    1        0        0  
4         1       0     0   1    0        0        0  

[5 rows x 21 columns]

## 从客舱号中提取客舱类别

In [26]:
cabinDf = pd.DataFrame()
full['Cabin'] = full['Cabin'].map(lambda c : c[0])
cabinDf = pd.get_dummies(full['Cabin'], prefix = 'Cabin')
cabinDf.head()

Cabin_A  Cabin_B  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  \
0        0        0        0        0        0        0        0        0   
1        0        0        1        0        0        0        0        0   
2        0        0        0        0        0        0        0        0   
3        0        0        1        0        0        0        0        0   
4        0        0        0        0        0        0        0        0   

   Cabin_U  
0        1  
1        0  
2        1  
3        0  
4        1

In [27]:
full = pd.concat([full, cabinDf], axis = 1)
full.drop('Cabin', axis = 1, inplace = True)
full.head()

Age     Fare  Parch  PassengerId  Sex  SibSp  Survived            Ticket  \
0  22.0   7.2500      0            1    1      1       0.0         A/5 21171   
1  38.0  71.2833      0            2    0      1       1.0          PC 17599   
2  26.0   7.9250      0            3    0      0       1.0  STON/O2. 3101282   
3  35.0  53.1000      0            4    0      1       1.0            113803   
4  35.0   8.0500      0            5    1      0       0.0            373450   

   Embarked_C  Embarked_Q   ...     Royalty  Cabin_A  Cabin_B  Cabin_C  \
0           0           0   ...           0        0        0        0   
1           1           0   ...           0        0        0        1   
2           0           0   ...           0        0        0        0   
3           0           0   ...           0        0        0        1   
4           0           0   ...           0        0        0        0   

   Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  Cabin_U  
0        0        0        0        0        0        1  
1        0        0        0        0        0        0  
2        0        0        0        0        0        1  
3        0        0        0        0        0        0  
4        0        0        0        0        0        1  

[5 rows x 29 columns]

## 建立家庭人数和家庭类别

In [28]:
familyDf = pd.DataFrame()
familyDf['FamilySize'] = full['Parch'] + full['SibSp'] + 1
familyDf['Family_Single'] = familyDf['FamilySize'].map(lambda s : 1 if s==1 else 0)
familyDf['Family_Small'] = familyDf['FamilySize'].map(lambda s : 1 if 2 <= s <= 4 else 0)
familyDf['Family_Large'] = familyDf['FamilySize'].map(lambda s : 1 if 5 <= s else 0)
familyDf.head()

FamilySize  Family_Single  Family_Small  Family_Large
0           2              0             1             0
1           2              0             1             0
2           1              1             0             0
3           2              0             1             0
4           1              1             0             0

In [29]:
full = pd.concat([full, familyDf], axis = 1)
full.drop(['Parch', 'SibSp'], axis = 1, inplace = True)
full.head()

Age     Fare  PassengerId  Sex  Survived            Ticket  Embarked_C  \
0  22.0   7.2500            1    1       0.0         A/5 21171           0   
1  38.0  71.2833            2    0       1.0          PC 17599           1   
2  26.0   7.9250            3    0       1.0  STON/O2. 3101282           0   
3  35.0  53.1000            4    0       1.0            113803           0   
4  35.0   8.0500            5    1       0.0            373450           0   

   Embarked_Q  Embarked_S  Pclass_1      ...       Cabin_D  Cabin_E  Cabin_F  \
0           0           1         0      ...             0        0        0   
1           0           0         1      ...             0        0        0   
2           0           1         0      ...             0        0        0   
3           0           1         1      ...             0        0        0   
4           0           1         0      ...             0        0        0   

   Cabin_G  Cabin_T  Cabin_U  FamilySize  Family_Single  Family_Small  \
0        0        0        1           2              0             1   
1        0        0        0           2              0             1   
2        0        0        1           1              1             0   
3        0        0        0           2              0             1   
4        0        0        1           1              1             0   

   Family_Large  
0             0  
1             0  
2             0  
3             0  
4             0  

[5 rows x 31 columns]

#### 累计特征：
年龄， 票价， 性别， 进港口， 舱号， 客舱等级， 家庭人数和类型

### 3.3 特征选择

In [32]:
corrDf = full.corr()

In [33]:
corrDf

Age      Fare  PassengerId       Sex  Survived  \
Age            1.000000  0.171521     0.025731  0.057397 -0.070323   
Fare           0.171521  1.000000     0.031416 -0.185484  0.257307   
PassengerId    0.025731  0.031416     1.000000  0.013406 -0.005007   
Sex            0.057397 -0.185484     0.013406  1.000000 -0.543351   
Survived      -0.070323  0.257307    -0.005007 -0.543351  1.000000   
Embarked_C     0.076179  0.286241     0.048101 -0.066564  0.168240   
Embarked_Q    -0.012718 -0.130054     0.011585 -0.088651  0.003650   
Embarked_S    -0.059153 -0.169894    -0.049836  0.115193 -0.149683   
Pclass_1       0.362587  0.599956     0.026495 -0.107371  0.285904   
Pclass_2      -0.014193 -0.121372     0.022714 -0.028862  0.093349   
Pclass_3      -0.302093 -0.419616    -0.041544  0.116562 -0.322308   
Master        -0.363923  0.011596     0.002254  0.164375  0.085221   
Miss          -0.254146  0.092051    -0.050027 -0.672819  0.332795   
Mr             0.165476 -0.192192     0.014116  0.870678 -0.549199   
Mrs            0.198091  0.139235     0.033299 -0.571176  0.344935   
Officer        0.162818  0.028696     0.002231  0.087288 -0.031316   
Royalty        0.059466  0.026214     0.004400 -0.020408  0.033391   
Cabin_A        0.125177  0.020094    -0.002831  0.047561  0.022287   
Cabin_B        0.113458  0.393743     0.015895 -0.094453  0.175095   
Cabin_C        0.167993  0.401370     0.006092 -0.077473  0.114652   
Cabin_D        0.132886  0.072737     0.000549 -0.057396  0.150716   
Cabin_E        0.106600  0.073949    -0.008136 -0.040340  0.145321   
Cabin_F       -0.072644 -0.037567     0.000306 -0.006655  0.057935   
Cabin_G       -0.085977 -0.022857    -0.045949 -0.083285  0.016040   
Cabin_T        0.032461  0.001179    -0.023049  0.020558 -0.026456   
Cabin_U       -0.271918 -0.507197     0.000208  0.137396 -0.316912   
FamilySize    -0.196996  0.226465    -0.031437 -0.188583  0.016639   
Family_Single  0.116675 -0.274826     0.028546  0.284537 -0.203367   
Family_Small  -0.038189  0.197281     0.002975 -0.255196  0.279855   
Family_Large  -0.161210  0.170853    -0.063415 -0.077748 -0.125147   

               Embarked_C  Embarked_Q  Embarked_S  Pclass_1  Pclass_2  \
Age              0.076179   -0.012718   -0.059153  0.362587 -0.014193   
Fare             0.286241   -0.130054   -0.169894  0.599956 -0.121372   
PassengerId      0.048101    0.011585   -0.049836  0.026495  0.022714   
Sex             -0.066564   -0.088651    0.115193 -0.107371 -0.028862   
Survived         0.168240    0.003650   -0.149683  0.285904  0.093349   
Embarked_C       1.000000   -0.164166   -0.778262  0.325722 -0.134675   
Embarked_Q      -0.164166    1.000000   -0.491656 -0.166101 -0.121973   
Embarked_S      -0.778262   -0.491656    1.000000 -0.181800  0.196532   
Pclass_1         0.325722   -0.166101   -0.181800  1.000000 -0.296526   
Pclass_2        -0.134675   -0.121973    0.196532 -0.296526  1.000000   
Pclass_3        -0.171430    0.243706   -0.003805 -0.622172 -0.563180   
Master          -0.014172   -0.009091    0.018297 -0.084504 -0.016933   
Miss            -0.014351    0.198804   -0.113886 -0.011733 -0.025440   
Mr              -0.065538   -0.080224    0.108924 -0.099725 -0.038595   
Mrs              0.098379   -0.100374   -0.022950  0.141102  0.071103   
Officer          0.003678   -0.003212   -0.001202  0.098788  0.073070   
Royalty          0.077213   -0.021853   -0.054250  0.118561 -0.035156   
Cabin_A          0.094914   -0.042105   -0.056984  0.228433 -0.067736   
Cabin_B          0.161595   -0.073613   -0.095790  0.399378 -0.118426   
Cabin_C          0.158043   -0.059151   -0.101861  0.485974 -0.144104   
Cabin_D          0.107782   -0.061459   -0.056023  0.275698 -0.037929   
Cabin_E          0.027566   -0.042877    0.002960  0.242963 -0.050210   
Cabin_F         -0.020010   -0.020282    0.030575 -0.073083  0.127371   
Cabin_G         -0.031566   -0.019941    0.040560 -0.035441 -0.032081   
Cabin_T         -0.01409

In [37]:
corrDf['Survived'].sort_values(ascending = False)

Survived         1.000000
Mrs              0.344935
Miss             0.332795
Pclass_1         0.285904
Family_Small     0.279855
Fare             0.257307
Cabin_B          0.175095
Embarked_C       0.168240
Cabin_D          0.150716
Cabin_E          0.145321
Cabin_C          0.114652
Pclass_2         0.093349
Master           0.085221
Cabin_F          0.057935
Royalty          0.033391
Cabin_A          0.022287
FamilySize       0.016639
Cabin_G          0.016040
Embarked_Q       0.003650
PassengerId     -0.005007
Cabin_T         -0.026456
Officer         -0.031316
Age             -0.070323
Family_Large    -0.125147
Embarked_S      -0.149683
Family_Single   -0.203367
Cabin_U         -0.316912
Pclass_3        -0.322308
Sex             -0.543351
Mr              -0.549199
Name: Survived, dtype: float64

In [39]:
#特征选择
full_X = pd.concat( [titleDf,#头衔
                     pclassDf,#客舱等级
                     familyDf,#家庭大小
                     full['Fare'],#船票价格
                     cabinDf,#船舱号
                     embarkedDf,#登船港口
                     full['Sex']#性别
                    ] , axis=1 )
full_X.head()

Master  Miss  Mr  Mrs  Officer  Royalty  Pclass_1  Pclass_2  Pclass_3  \
0       0     0   1    0        0        0         0         0         1   
1       0     0   0    1        0        0         1         0         0   
2       0     1   0    0        0        0         0         0         1   
3       0     0   0    1        0        0         1         0         0   
4       0     0   1    0        0        0         0         0         1   

   FamilySize ...   Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  Cabin_U  \
0           2 ...         0        0        0        0        0        1   
1           2 ...         0        0        0        0        0        0   
2           1 ...         0        0        0        0        0        1   
3           2 ...         0        0        0        0        0        0   
4           1 ...         0        0        0        0        0        1   

   Embarked_C  Embarked_Q  Embarked_S  Sex  
0           0           0           1    1  
1           1           0           0    0  
2           0           0           1    0  
3           0           0           1    0  
4           0           0           1    1  

[5 rows x 27 columns]

# 构建模型

In [41]:
sourceRow = 891
source_X = full_X.loc[0 : sourceRow-1, :]
source_y = full.loc[0 : sourceRow-1, 'Survived']
pre_X = full_X.loc[sourceRow: , :]

In [44]:
source_X.shape[0]
#pre_X.shape[0]

891

In [47]:
from sklearn.cross_validation import train_test_split 
train_X, test_X, train_y, test_y = train_test_split(source_X ,
                                                    source_y,
                                                    train_size=.8)

## 选择一个机器学习算法：逻辑回归

In [49]:
#第1步：导入算法
from sklearn.linear_model import LogisticRegression
#第2步：创建模型：逻辑回归（logisic regression）
model = LogisticRegression()
#第3步：训练模型
model.fit( train_X , train_y )
model.score(test_X , test_y )

0.8547486033519553

In [52]:
pred_Y = model.predict(pre_X)
pred_Y = pred_Y.astype(int)
passenger_id = full.loc[sourceRow:,'PassengerId']
predDf = pd.DataFrame( 
    { 'PassengerId': passenger_id , 
     'Survived': pred_Y } )
predDf.shape
predDf.head()

PassengerId  Survived
891          892         0
892          893         1
893          894         0
894          895         0
895          896         1

In [53]:
predDf.to_csv( 'titanic_pred.csv' , index = False )